In [ ]:
# @title Setup videos for running predinction on it prediction

# Create the SampleVideos folder if it doesn't exist
!mkdir -p SampleVideos

# Download the videos using wget
!wget -O SampleVideos/video_1.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/1.mp4"
!wget -O SampleVideos/video_2.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/2.mp4"
!wget -O SampleVideos/video_3.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/3.mp4"
!wget -O SampleVideos/video_4.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/4.mp4"
!wget -O SampleVideos/video_5.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/5.mp4"
!wget -O SampleVideos/video_6.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/6.mp4"
!wget -O SampleVideos/video_7.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/7.mp4"
!wget -O SampleVideos/video_8.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/8.mp4"
!wget -O SampleVideos/video_9.mp4 "https://github.com/Senthilsk10/my-academic-files/raw/main/ecosense/videos/9.mp4"


In [ ]:
# @title Copying required yolov5 detection helper files from github

! git clone https://github.com/Senthilsk10/Ecosense.git
%cd /content/Ecosense/yolov5
!pip install -r requirements.txt

In [ ]:
# @title Helper function to extract frames and store it to  a folder.

import cv2
import os

def extract_frames(video_path, output_folder, fps_limit):
    # Read the video from the specified path
    cam = cv2.VideoCapture(video_path)

    try:
        # Creating a folder named data
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    # If not created, then raise an error
    except OSError:
        print('Error: Creating directory of', output_folder)

    # Frame and seconds in video
    current_frame = 0
    current_second = 0
    frame_dict = {}

    # Get the frames per second of the input video
    fps = cam.get(cv2.CAP_PROP_FPS)

    # Calculate the frame interval based on the desired fps limit
    frame_interval = int(fps / fps_limit)

    while True:
        # Reading a frame
        ret, frame = cam.read()

        if ret:
            # If video is still left, continue creating images
            if current_frame % frame_interval == 0:
                # Create filename based on frame number
                filename = f"{current_frame // frame_interval}.png"
                # print('Creating...', filename)

                # Writing the extracted images
                cv2.imwrite(os.path.join(output_folder, filename), frame)

                # Store frame number and seconds in video in dictionary
                frame_dict[filename] = round(current_second,2)

            # Increasing counter
            current_frame += 1
            current_second += 1 / fps
        else:
            break

    # Release all space and windows once done
    cam.release()
    cv2.destroyAllWindows()

    return frame_dict

# Set your desired frames per second limit
# video_path = "/content/drive/My Drive/ecosense/3.mp4"
# fps_limit = 5
# output_folder = "/content/images"
# frame_dict = extract_frames(video_path, output_folder, fps_limit)

# print(frame_dict)


In [ ]:
# @title Helper function to run inference on images stored on a folder using commandline


def run_detection(weights, img_size, conf_thres, source):
    """
    Run YOLOv5 detection with specified arguments.

    Args:
        weights (str): Path to the model weights file.
        img_size (int): Input image size for inference.
        conf_thres (float): Confidence threshold for detections.
        source (str): Source path for images/videos.

    Returns:
        str: Captured output of the detection process.
    """

    command = f"python detect.py --weights {weights} --img {img_size} --conf {conf_thres} --source {source} --save-txt"
    output = !{command}
    return '\n'.join(output)

#Example usage
# captured_output = run_detection(
#     weights="runs/train/yolov5s_results/weights/best.pt",
#     img_size=416,
#     conf_thres=0.4,
#     source="/content/images"
# )
# print("Captured Output:")
# print(captured_output)


In [ ]:
# @title helper functions for counting wastes in the given frame.
import re
def get_waste_count(captured_output):
    # Dictionary to store frame numbers and corresponding wastecount
    frame_dict = {}

    # Regex pattern to match image names and wastecount values
    pattern = r"image (\d+)/\d+ (/content/images/\d+)\.png:.*?(\d+) waste"

    # Iterate through each line of the output
    for line in captured_output.splitlines():
        match = re.search(pattern, line)
        if match:
            image_name = match.group(2)
            image_number = int(image_name.split("/")[-1])
            wastecount = int(match.group(3)) if match.group(3) else 0
            frame_dict[int(image_number)] = wastecount
    max_key = max(frame_dict.keys()) if frame_dict else 0

    for i in range(max_key + 1):
        if i not in frame_dict:
            frame_dict[i] = 0
    sorted_frame_dict = dict(sorted(frame_dict.items()))
    # Return the dictionary containing frame numbers and corresponding wastecount
    # print(sorted_frame_dict)
    return sorted_frame_dict


# results = get_waste_count(captured_output)
# print(results)

In [ ]:
# @title helper functions for monitoring changes from predicted results of the video frames.

def monitor_changes(waste_results):
    prev = 0
    changes = []
    for key, val in waste_results.items():
        if val > prev and waste_results[key + 1] == val:
            changes.append(key)
        prev = val
    #print(changes)
    return changes

# To get the clip timings of the video
def get_clip_list(changes):
  clip_list = []

  for t in changes:
    start_time = frame_dict.get(f"{t-1}.png", 0)
    end_time = frame_dict.get(f"{t+2}.png", 0)
    clip_list.append([start_time, end_time])

  return clip_list

# changes = monitor_changes(results)
# clip_list = get_clip_list(changes)
# print(clip_list)


In [ ]:
# @title Helper function to sow the outputs of video where people put wastes

from IPython.display import HTML

def create_popup_html(message, time_seconds, youtube_embed_link):
    # Create the list of YouTube video iframes with specified start and stop times
    iframe_list = ''
    for time_sec in time_seconds:
        stop_time = time_sec + 3  # Adding 3 seconds to the start time for the stop time
        iframe_src = f"{youtube_embed_link}&start={time_sec}&end={stop_time}&controls=0&modestbranding=1&showinfo=0&fs=0&loop=1&rel=0"
        iframe_list += f'''
        <div>
            <h3>Start Time: {time_sec} seconds | Stop Time: {stop_time} seconds</h3>
            <iframe width="560" height="315" src="{iframe_src}" title="YouTube video player" frameborder="0" allowfullscreen></iframe>
        </div>
        '''

    # HTML code for the popup
    popup_html = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Popup Example</title>
        <style>
            .video-container {{
                margin-bottom: 20px;
            }}
        </style>
    </head>
    <body>
        <h1>{message}</h1>
        <div class="video-container">
            {iframe_list}
        </div>
    </body>
    </html>
    '''

    return popup_html,iframe_list

# Variables
#message = "Check out these videos!"
#time_seconds = [0, 30, 60]  # List of start times in seconds
#youtube_embed_link = "https://www.youtube.com/embed/EUMpUUXKvP0?si=xjEdA0SPz9yG-0vE"
#
## Create the popup HTML content
#popup_html = create_popup_html(message, time_seconds, youtube_embed_link)
#
## Display the HTML output
#HTML(popup_html)


In [ ]:
# @title Displaying all available smaple videos to test

# Dictionary of YouTube video links
links = {
    "1": "https://www.youtube.com/embed/JcZp19KPFYo?si=PrducC6gYMRG1BQB",
    "2": "https://www.youtube.com/embed/Qh_CxrIOTX4?si=XryIUsDKN4sKGPQE",
    "3": "https://www.youtube.com/embed/Vlz8kAkIzck?si=gtxl5XV68VOoIJgc",
    "4": "https://www.youtube.com/embed/nXmwzHg2Fso?si=oLgWhoMSD_wfNvJS",
    "5": "https://www.youtube.com/embed/_x36PCsU7bM?si=Xna2kQi57Ws01OWm",
    "6": "https://www.youtube.com/embed/fWpDozeJ2_k?si=NfrlTR_09E34QL0T",
    "7": "https://www.youtube.com/embed/3uB4WC4ohw8?si=JesF2DUNS8kiai_-",
    "8": "https://www.youtube.com/embed/n234_yjAAb4?si=OXsjPfE_N5nPsJJ0",
    "9": "https://www.youtube.com/embed/SjTt-563PnU?si=MADiutAu9a9P-H7c"
}

# Create the HTML for the popup window
popup_html = '''
<!DOCTYPE html>
<html>
<head>
  <title>YouTube Videos Popup</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 20px;
    }
    .video-container {
      display: grid;
      grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
      grid-gap: 20px;
    }
    .video-iframe {
      width: 100%;
      height: 200px;
    }
    h1 {
      text-align:center;
    }
  </style>
</head>
<body>
  <h1>Select any one of the video from the below cell</h1>
  <div class="video-container">
'''

# Add each YouTube video iframe to the HTML
for video_id, video_link in links.items():
    iframe_code = f'''
    <div>
      <h3>Video {video_id}</h3>
      <iframe class="video-iframe" src="{video_link}" frameborder="0" allowfullscreen></iframe>
    </div>
    '''
    popup_html += iframe_code

# Close the HTML
popup_html += '''
  </div>
</body>
</html>
'''

# Display the HTML in a popup window
HTML(f'''
<button onclick="openPopup()">Open Popup</button>
<script>
function openPopup() {{
  var popup = window.open('', 'myPopup', 'width=800,height=600');
  popup.document.write(`{popup_html}`);
}}
</script>
''')


In [ ]:
# @title Main Function - Select any video from the following options
video_path = "/content/SampleVideos/video_2.mp4" # @param ["/content/SampleVideos/video_1.mp4", "/content/SampleVideos/video_2.mp4", "/content/SampleVideos/video_3.mp4", "/content/SampleVideos/video_4.mp4", "/content/SampleVideos/video_5.mp4", "/content/SampleVideos/video_6.mp4", "/content/SampleVideos/video_7.mp4", "/content/SampleVideos/video_8.mp4", "/content/SampleVideos/video_9.mp4"]
video_id = str(video_path[-5])
output_folder = "/content/images"
fps_limit = 5

frame_dict = extract_frames(video_path, output_folder, fps_limit)
video_len = frame_dict[f"{frame_dict.__len__() - 1}.png"]

captured_output = run_detection(
    weights="/content/Ecosense/yolov5/runs/train/yolov5s_results/weights/best.pt",
    img_size=416,
    conf_thres=0.4,
    source=output_folder
)

results = get_waste_count(captured_output)
changes = monitor_changes(results)
clip_list = get_clip_list(changes)

#print("Results for calculating the Wastes in video",results)
#print("Time in seconds where improper disposal found on the video",clip_list)


In [ ]:
# @title Displaying Results
from math import floor


embed_link = links[video_id]

if changes[0] == 0:
  message = "No improper disposal activity found\n"
  time_seconds = []
  popup_html = create_popup_html(message, time_seconds, embed_link)
else:
  time_seconds = [floor(i[0]) for i in clip_list]
  message = "Improper Waste detections found on the following key times"
  popup_html,iframe_list = create_popup_html(message, time_seconds, embed_link)

HTML(popup_html)


In [ ]:
# @title Use this to see  results of every step

print("Video Path:", video_path)
# Description: The path to the input video file.

print("Video ID:", video_id)
# Description: The ID of the video extracted from the file name.

print("Output Folder:", output_folder)
# Description: The folder where extracted frames will be saved.

print("FPS Limit:", fps_limit)
# Description: The frame rate limit for extracting frames from the video.

print("Frame Dictionary:", frame_dict)
# Description: A dictionary containing extracted frames with their file paths as keys.

print("Video Length:", video_len)
# Description: The length of the video in frames.

print("Captured Output:", captured_output)
# Description: The detection results obtained from running YOLOv5 on the extracted frames.

print("Detection Results:", results)
# Description: The count of waste items detected in the video.

print("Changes:", changes)
# Description: A list of frames where changes in waste items occur.

print("Clip List:", clip_list)
# Description: A list of time intervals where waste changes occur.

print("iframe_list : ", iframe_list)
#Description : Alist of embed links that might have been taken.

MIT License

Copyright (c) 2024 Senthil Kumaran

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.